In [1]:
import pandas as pd 
import numpy as np 
import os 
import pickle
import urllib
import requests
import json

In [2]:


import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [3]:
def url_cleaning(df):
    """
    
    """
    
    df = (df.assign(url_new = lambda x : x.url.str.split(":",expand=True)[1]) #remove http and https 
            .sort_values(by='pub_date', ascending=False, na_position='last')
            .drop_duplicates(subset='url_new', keep='first'))
    return df 

In [4]:
def vg_plus_cleaning(df):
    """
    clean vgs + related articles (subscription related articles)
    """
    df = df.drop(df.index[df.text_len<30])
    return df

In [5]:
def read_vg_file(file):
    """
    
    """
    #df = pd.read_json(json.dumps(pickle.load( open( data_folder+name, "rb" ) )))
    
    #data = pd.DataFrame()
    #pd.DataFrame.from_dict(pickle.load(open("../spiders/data/vg/20.03.01_18:56:02.pickle","rb")))
    df = (pd.DataFrame.from_dict(pickle.load(open(file,'rb')))
            .replace('None', np.nan)
            .assign(text_len= lambda x: x.text.str.len())
            .assign(pub_date=lambda x: pd.to_datetime(x['pub_date']))
            .dropna(axis=0,subset=['text'])
            .sort_values(by='pub_date',na_position='last')
            .pipe(url_cleaning)
            .pipe(vg_plus_cleaning)#test[test.x > 0].index
         )
    return df


In [6]:
folder = "../spiders/data/vg/"
name1 = "20.03.01_21:00:01.pickle"
data = read_vg_file(folder+name1)

orginalt: 34931 etter: 34468

In [7]:
def get_vg_data(folder):
    """
    
    """
    files = os.listdir(folder)
    df = pd.DataFrame(columns = ['url','text','title','pub_date'])
    for file in files:
        #read, sanity check, sort on date, remove duplicates
        # merge with exicting, sort on date, remove duplicates 
        pass
    

In [8]:
nlp = spacy.load("nb_core_news_sm")

name = '20.01.23 23:19:58.pickle'
name = "20.03.01_18:56:02.pickle"
data_folder = "../spiders/data/vg/"
with open(data_folder+name, 'rb') as handle:
    b = pickle.load(handle)

In [9]:
data.sample(3)

,text,title,url,pub_date,text_len,url_new
34999,"Det viser et dokument datert 27. mai 2016, som...",Sykkelpresidenten ga seg selv kredittkort-gara...,https://www.vg.no/sport/sykkel/i/J1Rwa6/sykkel...,2018-03-08 09:40:54,3958.0,//www.vg.no/sport/sykkel/i/J1Rwa6/sykkelpresid...
10533,Det sier høyprofilerte Rangappa i dette interv...,Tidligere FBI-agent om Trump: «Hvorfor lyve hv...,https://www.vg.no/nyheter/utenriks/i/21lrVr/ti...,2018-05-12 15:33:55,4444.0,//www.vg.no/nyheter/utenriks/i/21lrVr/tidliger...
16994,– Forutsigbarhet. Norsk fotball trenger foruts...,Snart får du vite når seriestarten blir - i 20...,https://www.vg.no/sport/fotball/i/vmzKdm/snart...,2018-03-10 18:37:53,1308.0,//www.vg.no/sport/fotball/i/vmzKdm/snart-faar-...


In [10]:
url_stop_ord = "https://gist.githubusercontent.com/kmelve/8869818/raw/407980a1d89a385283d738e1545ab1b2014e87be/stoppord_no"
response = requests.get(url_stop_ord,)
my_stop_words = response.content.decode("utf-8") .split("\n")

In [11]:
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [12]:
data['text'][:20].astype('unicode').apply(nlp)

0        (Ullevål, sykehus, skjerpet, søndag, sine, kar...
35033    (Folkehelseinstituttet, opplyser, at, tolv, av...
35011    (Folkehelseinstituttet, opplyser, at, tolv, av...
35012    ( , Myndighetene, i, Seoul, har, anmeldt, Lee,...
35009    (Nå, krever, Norges, største, parti, at, det, ...
30       (I, fjor, fortalte, politifolk, i, Kripos, til...
13       (En, konsekvens, av, Det, hører, med, i, dette...
26       (I, går, kveld, Andersen, tiltrådte, som, stat...
29       (–, Vi, har, sett, dette, tydelig, de, siste, ...
28       (Du, vinker, meg, til, side, i, korridoren, .,...
11       ( , Wold, bor, og, jobber, i, Istanbul, ,, men...
6        (17, personer, har, fått, påvist, coronaviruse...
14       (Det, kommer, frem, i, en, oppdatering, fra, O...
34       (Det, kommer, frem, i, en, oppdatering, fra, O...
51       (Det, kommer, frem, i, en, oppdatering, fra, O...
23       (Marcello, Longhi, er, prest, i, en, av, mange...
3        (–, Alle, de, 15, har, relasjon, til, utbredel.

In [13]:
data.text

0        Ullevål sykehus skjerpet søndag sine karantene...
35033    Folkehelseinstituttet opplyser at tolv av tilf...
35011    Folkehelseinstituttet opplyser at tolv av tilf...
35012     Myndighetene i Seoul har anmeldt Lee Man-hee,...
35009    Nå krever Norges største parti at det nedsette...
                               ...                        
5384     Lykkelig over at katten ikke ser ut til å ha n...
925      Jo, det er Tom Hanks. Juryen har talt.Det var ...
30096    Ifølge TV 2 har Nygaard nylig mottatt et brev ...
4562     Norske TV-seere har kunnet følge Dawson, spilt...
5366     Hun merket ikke at en av husets katter hadde l...
Name: text, Length: 34468, dtype: object

In [21]:
%%time
data['text'].sample(1000).apply(nlp)

CPU times: user 57.6 s, sys: 7.14 s, total: 1min 4s
Wall time: 1min 5s


12516    (Vertslandet, Norge, oppjusterer, også, antall...
32670    (Det, er, solgt, cirka, 22.000, billetter, til...
4385     (Det, statlige, nyhetsbyrået, SANA, skriver, a...
20452    (–, Jeg, synes, det, er, gøy, å, prøve, litt, ...
9073     (Belgia, hadde, lenge, problemer, med, å, bryt...
                               ...                        
10623    (Kilder, Ifølge, CNN, hevder, Cohen, å, ha, væ...
19644    (–, Vi, kunne, lent, oss, tilbake, og, gått, p...
27711    (–, Jeg, er, totalt, forandret, siden, sist, ....
16828    (Til, mandag, er, det, skolestart, over, hele,...
362      (I, en, Twitter, -, melding, natt, til, tirsda...
Name: text, Length: 1000, dtype: object

In [14]:
data['processes_text'] = pd.Series(np.nan, index=data.index)

In [15]:
%%time

for i, chunk in enumerate(np.array_split(data.index.values, 30)):
    print(f'fold {i}')
    data['processes_text'].loc[chunk] = data['text'].loc[chunk].apply(nlp)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
fold 11
fold 12
fold 13
fold 14
fold 15
fold 16
fold 17
fold 18
fold 19
fold 20
fold 21
fold 22
fold 23
fold 24
fold 25
fold 26
fold 27
fold 28
fold 29
CPU times: user 32min 37s, sys: 6min 56s, total: 39min 34s
Wall time: 40min 24s


In [ ]:
1586464

In [17]:
data.memory_usage(index=True).sum()/10**6

3.516672

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    """
    https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%time
lass = parallelize_dataframe(df.text,nlp)#train = parallelize_dataframe(train_df, add_features)


In [18]:
# we add some words to the stop word list
texts, article = [], []
for pd_article in data.processes_text:
    for w in pd_article: 
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            article.append(w.lemma_)
    texts.append(article)
    article = []



In [19]:
bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]


In [20]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

## LSI

In [21]:
lsimodel = LsiModel(corpus=corpus, num_topics=20, id2word=dictionary)
lsimodel.show_topics(num_topics=10)  # Showing only the top 5 topics


[(0,
  '0.292*"VG" + 0.253*" " + 0.158*"møte" + 0.155*"parti" + 0.154*"fortelle" + 0.124*"skje" + 0.123*"vite" + 0.107*"barn" + 0.106*"Trump" + 0.105*"snakke"'),
 (1,
  '0.393*"parti" + -0.332*"spiller" + -0.279*"spille" + 0.268*"KrF" + -0.190*"Solskjær" + 0.183*" " + -0.174*"klubbe" + 0.171*"Frp" + -0.154*"sesong" + -0.134*"kamp"'),
 (2,
  '-0.559*"Trump" + 0.372*"parti" + 0.245*"KrF" + 0.196*"spiller" + 0.172*"spille" + -0.148*"barn" + 0.145*"Frp" + -0.135*"president" + 0.118*"Solskjær" + -0.109*" "'),
 (3,
  '0.712*" " + -0.450*"Trump" + -0.230*"parti" + 0.151*"barn" + 0.127*"VG" + -0.113*"KrF" + -0.111*"møte" + -0.107*"president" + 0.091*"skole" + -0.074*"valg"'),
 (4,
  '-0.571*" " + 0.344*"barn" + -0.326*"Trump" + 0.297*"VG" + 0.229*"fortelle" + -0.147*"spiller" + -0.117*"spille" + -0.103*"Solskjær" + 0.099*"skole" + -0.098*"parti"'),
 (5,
  '0.604*"VG" + -0.523*"barn" + -0.183*"skole" + -0.144*"  " + 0.138*"møte" + -0.122*"parti" + 0.110*"fortelle" + -0.107*"små" + -0.093*"forel

## HDP

In [22]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdpmodel.show_topics()


[(0,
  '0.005*VG + 0.005*  + 0.003*fortelle + 0.003*møte + 0.003*parti + 0.002*skje + 0.002*spille + 0.002*vite + 0.002*spiller + 0.002*Trump + 0.002*legge + 0.002*dag + 0.002*snakke + 0.002*små + 0.002*uke + 0.002*jo + 0.002*følge + 0.002*holde + 0.002*egen + 0.002*svare'),
 (1,
  '0.005*VG + 0.004*barn + 0.004*møte + 0.003*  + 0.003*vite + 0.003*fortelle + 0.003*parti + 0.002*uke + 0.002*Trump + 0.002*skje + 0.002*egen + 0.002*jobbe + 0.002*snakke + 0.002*svare + 0.002*små + 0.002*støtte + 0.002*heller + 0.002*dag + 0.002*jobb + 0.002*jo'),
 (2,
  '0.004*avtale + 0.004*EU + 0.003*  + 0.003*Johnson + 0.003*statsminister + 0.003*Solskjær + 0.003*spiller + 0.003*Storbritannia + 0.002*små + 0.002*skje + 0.002*uke + 0.002*brexit + 0.002*egen + 0.002*spille + 0.002*dag + 0.002*jobb + 0.002*Manchester_United + 0.002*fortelle + 0.002*VG + 0.002*Boris_Johnson'),
 (3,
  '0.005*Trump + 0.004*VG + 0.003*Iran + 0.003*  + 0.003*fortelle + 0.003*barn + 0.002*møte + 0.002*stemme + 0.002*situasjon + 

## LDA

In [23]:
ldamodel = LdaModel(corpus=corpus, num_topics=20, id2word=dictionary)
ldamodel.show_topics()


[(6,
  '0.010*"jo" + 0.009*"låt" + 0.006*"spille" + 0.006*"skuespiller" + 0.005*"kjenne" + 0.005*"hund" + 0.005*"musikk" + 0.005*"fin" + 0.005*"alltid" + 0.005*"serie"'),
 (11,
  '0.024*"EU" + 0.018*"avtale" + 0.012*"valg" + 0.011*" " + 0.011*"parti" + 0.010*"politisk" + 0.009*"stemme" + 0.009*"statsminister" + 0.007*"dersom" + 0.006*"Storbritannia"'),
 (7,
  '0.037*"russisk" + 0.020*"Putin" + 0.014*"Syria" + 0.013*"IS" + 0.012*"russer" + 0.007*"Moskva" + 0.006*"myndighet" + 0.005*"sivil" + 0.005*"Tyrkia" + 0.005*"gruppe"'),
 (10,
  '0.010*" " + 0.009*"område" + 0.008*"fortelle" + 0.008*"dag" + 0.007*"uke" + 0.007*"skade" + 0.007*"melde" + 0.007*"fly" + 0.006*"bile" + 0.006*"VG"'),
 (9,
  '0.037*"Real_Madrid" + 0.017*"Rangers" + 0.013*"spansk" + 0.010*"Spania" + 0.010*"Madrid" + 0.009*"Ramos" + 0.007*"b" + 0.006*"sluttavtale" + 0.006*"Zidane" + 0.005*"utvisning"'),
 (12,
  '0.020*"film" + 0.012*"tv" + 0.008*" " + 0.008*"program" + 0.007*"TV" + 0.007*"deltager" + 0.007*"programleder" + 

## pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)